#EXTRACCIÓN DE CARACTERÍSTICAS Y CLASIFICACIÓN

#INSTALACIÓN DE LIBRERIAS



## Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Librerías genéricas

In [ ]:
!pip install beartype -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.2 MB/s eta 0:00:00


In [ ]:
!pip install torchsummary

In [ ]:
from itertools import product

In [ ]:
import numpy as np
import torch
import cv2
import scipy.misc
import scipy.ndimage
import scipy.io
import os
import pickle
from math import floor
import datetime
from PIL import Image, ImageDraw, ImageFont

import sys
import math
import shutil
import argparse
from pathlib import Path
import torch.nn as nn
import scipy.special
import matplotlib.pyplot as plt
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
SMALL_SIZE = 16
MEDIUM_SIZE = 19
BIGGER_SIZE = 24
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
# plt.rcParams["figure.autolayout"] = True

from beartype import beartype
from scipy.io import loadmat
from tqdm import tqdm

import struct


<ipython-input-5-ef629bfbef8b>:4: DeprecationWarning: scipy.misc is deprecated and will be removed in 2.0.0
  import scipy.misc


In [ ]:
#from webvtt import WebVTT, Caption
#from zsvision.zs_utils import BlockTimer

# HELPER FUNCTIONS

## Funciones para conversión y tratamiento de frames

In [ ]:
def torch_to_list(torch_tensor):
    return torch_tensor.cpu().numpy().tolist()

def save_pred(preds, checkpoint="checkpoint", filename="preds_valid.mat"):
    preds = to_numpy(preds)
    checkpoint.mkdir(exist_ok=True, parents=True)
    filepath = os.path.join(checkpoint, filename)
    mdict = {"preds": preds}
    print(f"Saving to {filepath}")
    scipy.io.savemat(filepath, mdict=mdict, do_compression=False, format="4")

def to_torch(ndarray):
    if type(ndarray).__module__ == "numpy":
        return torch.from_numpy(ndarray)
    elif not torch.is_tensor(ndarray):
        raise ValueError(f"Cannot convert {type(ndarray)} to torch tensor")
    return ndarray

def to_numpy(tensor):
    if torch.is_tensor(tensor):
        return tensor.cpu().numpy()
    elif type(tensor).__module__ != "numpy":
        raise ValueError(f"Cannot convert {type(tensor)} to numpy array")
    return tensor

def im_to_numpy(img):
    img = to_numpy(img)
    img = np.transpose(img, (1, 2, 0))  # H*W*C
    return img


def im_to_torch(img):
    img = np.transpose(img, (2, 0, 1))  # C*H*W
    img = to_torch(img).float()
    if img.max() > 1:
        img /= 255
    return img


def resize_generic(img, oheight, owidth, interp="bilinear", is_flow=False):
    """
    Args
    inp: numpy array: RGB image (H, W, 3) | video with 3*nframes (H, W, 3*nframes)
          |  single channel image (H, W, 1) | -- not supported:  video with (nframes, 3, H, W)
    """

    # resized_image = cv2.resize(image, (100, 50))
    ht, wd, chn = img.shape[0], img.shape[1], img.shape[2]
    if chn == 1:
        resized_img = scipy.misc.imresize(
            img.squeeze(), [oheight, owidth], interp=interp, mode="F"
        ).reshape((oheight, owidth, chn))
    elif chn == 3:
        # resized_img = scipy.misc.imresize(img, [oheight, owidth], interp=interp)  # mode='F' gives an error for 3 channels
        resized_img = cv2.resize(img, (owidth, oheight))  # inverted compared to scipy
    elif chn == 2:
        # assert(is_flow)
        resized_img = np.zeros((oheight, owidth, chn), dtype=img.dtype)
        for t in range(chn):
            # resized_img[:, :, t] = scipy.misc.imresize(img[:, :, t], [oheight, owidth], interp=interp)
            # resized_img[:, :, t] = scipy.misc.imresize(img[:, :, t], [oheight, owidth], interp=interp, mode='F')
            # resized_img[:, :, t] = np.array(Image.fromarray(img[:, :, t]).resize([oheight, owidth]))
            resized_img[:, :, t] = scipy.ndimage.interpolation.zoom(
                img[:, :, t], [oheight, owidth]
            )
    else: # Resizing for multi-channel image (e.g., video frames)
        in_chn = 3
        # Workaround, would be better to pass #frames
        if chn == 16:
            in_chn = 1
        if chn == 32:
            in_chn = 2
        nframes = int(chn / in_chn)#calculates the number of frames (nframes) by dividing the total number of channels (chn) by the number of channels per frame (in_chn)
        img = img.reshape(img.shape[0], img.shape[1], in_chn, nframes)
        resized_img = np.zeros((oheight, owidth, in_chn, nframes), dtype=img.dtype)
        for t in range(nframes):
            frame = img[:, :, :, t]  # img[:, :, t*3:t*3+3]
            frame = cv2.resize(frame, (owidth, oheight)).reshape(
                oheight, owidth, in_chn
            )
            # frame = scipy.misc.imresize(frame, [oheight, owidth], interp=interp)
            resized_img[:, :, :, t] = frame
        resized_img = resized_img.reshape(
            resized_img.shape[0], resized_img.shape[1], chn
        )

    if is_flow:
        # print(oheight / ht)
        # print(owidth / wd)
        resized_img = resized_img * oheight / ht
    return resized_img


def color_normalize(x, mean, std):
    """Normalize a tensor of images by subtracting (resp. dividing) by the mean (resp.
    std. deviation) statistics of a dataset in RGB space.
    """
    if x.dim() in {3, 4}:
        if x.size(0) == 1:#assumes a single-channel image
            x = x.repeat(3, 1, 1)#replicate the channel to have 3
        assert x.size(0) == 3, "For single video format, expected RGB along first dim"#--> 3 channels
        for t, m, s in zip(x, mean, std):#iterates over the three channels, subtracting the mean and dividing by the standard deviation in-place
            t.sub_(m)
            t.div_(s)
    elif x.dim() == 5:
        assert (
            x.shape[1] == 3
        ), "For batched video format, expected RGB along second dim" #collection of video sequences organized into a batch
        x[:, 0].sub_(mean[0]).div_(std[0])
        x[:, 1].sub_(mean[1]).div_(std[1])
        x[:, 2].sub_(mean[2]).div_(std[2])
    return x

def get_labels_start_end_time(frame_wise_labels, bg_class=["Sign"]):
    """get list of start and end times of each interval/ segment.

    Args:
        frame_wise_labels: list of framewise labels/ predictions.
        bg_class: list of all classes in frame_wise_labels which should be ignored

    Returns:
        labels: list of labels of the segments
        starts: list of start times of the segments
        ends: list of end times of the segments
    """
    labels = []
    starts = []
    ends = []
    last_label = frame_wise_labels[0]
    if frame_wise_labels[0] not in bg_class:
        labels.append(frame_wise_labels[0])
        starts.append(0)
    for i in range(len(frame_wise_labels)):
        if frame_wise_labels[i] != last_label:
            if frame_wise_labels[i] not in bg_class:
                labels.append(frame_wise_labels[i])
                starts.append(i)
            if last_label not in bg_class:
                ends.append(i)
            last_label = frame_wise_labels[i]
    if last_label not in bg_class:
        ends.append(i + 1)
    return labels, starts, ends

@beartype
def load_rgb_video(video_path: Path, fps: int) -> torch.Tensor:
    """
    Load frames of a video using cv2.
    """
    cap = cv2.VideoCapture(str(video_path))
    cap_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    cap_fps = cap.get(cv2.CAP_PROP_FPS)

    # cv2 won't be able to change frame rates for all encodings, so we use ffmpeg
    if cap_fps != fps:
        tmp_video_path = f"{video_path}.tmp.{video_path.suffix}"
        shutil.move(video_path, tmp_video_path)
        cmd = (f"ffmpeg -i {tmp_video_path} -pix_fmt yuv420p "
               f"-filter:v fps=fps={fps} {video_path}")
        print(f"Generating new copy of video with frame rate {fps}")
        os.system(cmd)
        Path(tmp_video_path).unlink()
        cap = cv2.VideoCapture(str(video_path))
        cap_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        cap_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        cap_fps = cap.get(cv2.CAP_PROP_FPS)
        assert cap_fps == fps, f"ffmpeg failed to produce a video at {fps}"

    f = 0
    rgb = []
    while True:
        # frame: BGR, (h, w, 3), dtype=uint8 0..255
        ret, frame = cap.read()
        if not ret:
            break
        # BGR (OpenCV) to RGB (Torch)
        frame = frame[:, :, [2, 1, 0]]
        rgb_t = im_to_torch(frame)
        rgb.append(rgb_t)
        f += 1
    cap.release()
    # (nframes, 3, cap_height, cap_width) => (3, nframes, cap_height, cap_width)
    rgb = torch.stack(rgb).permute(1, 0, 2, 3)
    print(f"Loaded video {video_path} with {f} frames [{cap_height}hx{cap_width}w] res. "
          f"at {cap_fps}")
    return rgb

@beartype
def prepare_input(
    rgb: torch.Tensor,
    resize_res: int = 256,
    inp_res: int = 224,
    mean: torch.Tensor = 0.5 * torch.ones(3), std=1.0 * torch.ones(3),
):
    """
    Process the video:
    1) Resize to [resize_res x resize_res]
    2) Center crop with [inp_res x inp_res]
    3) Color normalize using mean/std
    """
    iC, iF, iH, iW = rgb.shape
    rgb_resized = np.zeros((iF, resize_res, resize_res, iC))
    for t in range(iF):
        tmp = rgb[:, t, :, :]
        tmp = resize_generic(
            im_to_numpy(tmp), resize_res, resize_res, interp="bilinear", is_flow=False
        )
        rgb_resized[t] = tmp
    rgb = np.transpose(rgb_resized, (3, 0, 1, 2))
    # Center crop coords
    ulx = int((resize_res - inp_res) / 2)
    uly = int((resize_res - inp_res) / 2)
    # Crop 256x256
    rgb = rgb[:, :, uly : uly + inp_res, ulx : ulx + inp_res]
    rgb = to_torch(rgb).float()
    assert rgb.max() <= 1
    rgb = color_normalize(rgb, mean, std)
    return rgb

In [ ]:
@beartype
def sliding_windows(
        rgb: torch.Tensor,
        num_in_frames: int,
        stride: int,
) -> tuple:
    """
    Return sliding windows and corresponding (middle) timestamp
    """
    C, nFrames, H, W = rgb.shape
    # If needed, pad to the minimum clip length
    if nFrames < num_in_frames:
        rgb_ = torch.zeros(C, num_in_frames, H, W)
        rgb_[:, :nFrames] = rgb
        rgb_[:, nFrames:] = rgb[:, -1].unsqueeze(1)
        rgb = rgb_
        nFrames = rgb.shape[1]

    num_clips = math.ceil((nFrames - num_in_frames) / stride) + 1
    plural = ""
    if num_clips > 1:
        plural = "s"
    print(f"{num_clips} clip{plural} resulted from sliding window processing.")

    rgb_slided = torch.zeros(num_clips, 3, num_in_frames, H, W)
    t_mid = []
    # For each clip
    for j in range(num_clips):
        # Check if num_clips becomes 0
        actual_clip_length = min(num_in_frames, nFrames - j * stride)
        if actual_clip_length == num_in_frames:
            t_beg = j * stride
        else:
            t_beg = nFrames - num_in_frames
        t_mid.append(t_beg + num_in_frames / 2)
        rgb_slided[j] = rgb[:, t_beg : t_beg + num_in_frames, :, :]

        clip=rgb_slided[j]



        for i, frame in enumerate(clip):
            frame_uint8 = (frame * 255).byte().numpy().transpose(1, 2, 0)[:,:,:3]
            frame_filename = Path("/content/drive/MyDrive/Tesis/videos_1") / f"clip_{j}_frame_{i}.png"
            cv2.imwrite(str(frame_filename), frame_uint8)

    return rgb_slided, np.array(t_mid)


# Clasificación MSTCN

### Helper functions

In [ ]:
#@title utils.py

!pip install pympi-ling -q
import pympi
import sys
import os
import datetime
import pickle

def get_labels_start_end_time(frame_wise_labels, bg_class=["Sign"]):
    """get list of start and end times of each interval/ segment.

    Args:
        frame_wise_labels: list of framewise labels/ predictions.
        bg_class: list of all classes in frame_wise_labels which should be ignored

    Returns:
        labels: list of labels of the segments
        starts: list of start times of the segments
        ends: list of end times of the segments
    """
    labels = []
    starts = []
    ends = []
    last_label = frame_wise_labels[0]
    if frame_wise_labels[0] not in bg_class:
        labels.append(frame_wise_labels[0])
        starts.append(0)
    for i in range(len(frame_wise_labels)):
        if frame_wise_labels[i] != last_label:
            if frame_wise_labels[i] not in bg_class:
                labels.append(frame_wise_labels[i])
                starts.append(i)
            if last_label not in bg_class:
                ends.append(i)
            last_label = frame_wise_labels[i]
    if last_label not in bg_class:
        ends.append(i + 1)
    return labels, starts, ends


def torch_to_list(torch_tensor):
    return torch_tensor.cpu().numpy().tolist()


def get_num_signs(gt):
    """count number of signs.

    Args:
        gt: list of framewise labels/ predictions.

    Returns:
        number of signs
    """
    item_old = gt[0]
    count = 0
    for ix, item in enumerate(gt):
        if item_old == 0 and item == 1:
            count += 1
        if ix == len(gt)-1 and item != 1:
            count += 1

        item_old = item
    return count


def parse_glosses(anno_dir, target_tiers):
    """Parse ELAN EAF files into a python dictionary.

    Args:
        anno_dir: the root directory containing all eaf files to be parsed.
        target_tiers: the names of the tiers (e.g. "RH-IDgloss") to be parsed.

    Returns:
        the parsed data
    """
    anno_paths = list(anno_dir.glob("*.eaf"))
    print(f"Found {len(anno_paths)} in {anno_dir}")
    times = []
    count = 0
    end_old = 0
    fps = 25
    for anno_path in anno_paths:
        eafob = pympi.Elan.Eaf(str(anno_path))
        for tier in target_tiers:
            for annotation in eafob.get_annotation_data_for_tier(tier):
                start, end = [(x) / 1000 for x in annotation[:2]]
                if end_old == 0:
                    end_old = round(end*fps)+1
                if round(start*fps) - end_old < 6:
                    while end_old < round(start*fps):
                        times.append(end_old)
                        end_old += 1
                times.append(round(start*fps))
                times.append(round(end*fps))
                end_old = round(end*fps)+1

    return times


def get_weights(gt_dict):
    """Calculate weights for weighted cross entropy loss.

    Args:
        gt_dict: dictionary of the gt labels.

    Returns:
        list of weights per class
    """
    count_list = [0, 0]
    for _, item in gt_dict.items():
        item = list(item)
        count_list[1] += item.count(1)
        count_list[0] += item.count(0)

    weights = [1 / count_list[i] if count_list[i] != 0 else 1 for i in range(2)]
    weights_norm = [i / sum(weights) for i in weights]
    return weights_norm


def save_args(args, save_folder, opt_prefix="opt"):
    """Save arguments as .txt and .pkl file.

    Args:
        args: parser arguments
        save_folder: path to folder
        opt_prefix: name of file
    """
    opts = vars(args)
    os.makedirs(save_folder, exist_ok=True)

    # Save to text
    opt_filename = f"{opt_prefix}.txt"
    opt_path = os.path.join(save_folder, opt_filename)
    with open(opt_path, "a") as opt_file:
        opt_file.write("====== Options ======\n")
        for k, v in sorted(opts.items()):
            opt_file.write(f"{str(k)}: {str(v)}\n")
        opt_file.write("=====================\n")
        opt_file.write(f"launched at {str(datetime.datetime.now())}\n")

    # Save as pickle
    opt_picklename = f"{opt_prefix}.pkl"
    opt_picklepath = os.path.join(save_folder, opt_picklename)
    with open(opt_picklepath, "wb") as opt_file:
        pickle.dump(opts, opt_file)
    print(f"Saved options to {opt_path}")


####### Nuevaaaa
def create_folders(args):
    """Creates folder structure to save models/ results in Google Drive (for Google Colab).

    Args:
        args: Config object with model parameters.

    Returns:
        model_load_dir: Path to pretrained model (if specified, otherwise empty)
        model_save_dir: Path to the folder where the model is saved
        results_save_dir: Path to the folder where the (inference) results are saved
    """
    num_stages = args.num_stages
    num_layers = args.num_layers
    num_f_maps = args.num_f_maps
    features_dim = args.features_dim
    bz = args.bz
    lr = args.lr
    num_epochs = args.num_epochs
    train_data = args.train_data
    test_data = args.test_data
    repetition=args.repetition

    # Define Google Drive base path
    base_path = '/content/drive/MyDrive/Tesis'  # Update this to your desired directory in Google Drive

    # Define folder for saving model and results
    std_str = ''
    if args.regression:
        train_type = f'regression/std_{args.std}'
    else:
        train_type = 'classification'

    if args.weights is None:
        weighted_str = 'unweighted'
    elif args.weights == 'opt':
        weighted_str = 'weighted_opt'
    else:
        weighted_str = f'weighted_{args.weights[0]}_{args.weights[1]}'

    if args.feature_normalization == 1:
        norm_str = '_normalized'
    else:
        norm_str = ''

    # Load pretrained model from given path
    if args.pretrained:
        model_load_dir = args.pretrained
    else:
        if args.action == 'predict':
            model_load_dir = os.path.join(
                base_path,
                f"predict/"
                f"{num_stages}_{num_layers}_{num_f_maps}_{features_dim}_{bz}_{lr}_{weighted_str}"
                f"epoch-{str(args.extract_epoch)}.model"
            )
        else:
            model_load_dir = ''

    # Check if the model exists when in training or predicting mode
    if not os.path.exists(model_load_dir) and ((args.pretrained and args.action == 'train') or args.action == 'predict'):
        print(f'Pre-trained model does not exist at: {model_load_dir}')
        sys.exit()

    model_save_dir = os.path.join(
        base_path,
        args.folder,

        f"{num_stages}_{num_layers}_{num_f_maps}_{features_dim}_{bz}_{lr}_{weighted_str}_{repetition}_split{args.split_id}"
    )

    results_save_dir = os.path.join(
        base_path,args.folder,
        f"{num_stages}_{num_layers}_{num_f_maps}_{features_dim}_{bz}_{lr}_{weighted_str}_{repetition}_split{args.split_id}"
    )

    # Handle directory creation
    if args.action == 'train':
        if os.path.exists(model_save_dir) and not args.refresh:
            print(f'Model directory already exists: {model_save_dir}')
            # sys.exit()
        else:
            os.makedirs(model_save_dir, exist_ok=True)
            save_args(args, model_save_dir)

    return model_load_dir, model_save_dir, results_save_dir



In [ ]:
#@title viz.py

import os
from pathlib import Path
import numpy as np
import cv2
import torch
import torch.nn as nn

from bokeh.palettes import d3, Turbo256
import seaborn as sns

all_colors = []
for ii in range(22):
    all_colors += (
        d3["Category20"][20] + d3["Category20b"][20] + d3["Category20c"][20]
    )

def viz_results_paper(gt, pred, name='test', pred_prob=None, pred_actsig=None, pred_prob_actsig=None):
    fr_width = 30
    height = 420
    width = len(gt)*fr_width+240
    image = np.ones((height,width,3), np.uint8)*255

    palette = [(0.9,0.9,0.9), sns.color_palette("colorblind", 10)[0]]   # deep, muted, pastel, bright, dark, colorblind
    palette2 = [(0.9,0.9,0.9), sns.color_palette("pastel", 10)[0]]   # deep, muted, pastel, bright, dark, colorblind

    shift = 200
    item_old = gt[0]
    count = 1
    color = (0,0,255)
    cv2.putText(image,'gt',(5,150), cv2.FONT_HERSHEY_SIMPLEX, 5,(0,0,0),3)
    for ix, item in enumerate(gt):
        item_old = item
        color = tuple([palette[item][i] *255 for i in [2,1,0]])
        image[20:200, fr_width*(ix+1)+shift:fr_width*(ix+1)+shift-1+fr_width] = color

    item_old = pred[0]
    count = 1
    color = (0,0,255)
    cv2.putText(image,'ms',(5,350), cv2.FONT_HERSHEY_SIMPLEX, 5,(0,0,0),3)
    for ix, item in enumerate(pred):
        item_old = item
        color = tuple([palette2[item][i] *255 for i in [2,1,0]])
        image[220:400, fr_width*(ix+1)+shift:fr_width*(ix+1)+shift-1+fr_width] = color

    if pred_prob is not None:
        old_item = pred_prob[0]
        for ix, item in enumerate(pred_prob):
            y = 400-int(pred_prob[ix]*(400-220))
            x = fr_width*(ix+1)+shift+fr_width//2
            if ix > 0:
                image = cv2.line(image, (x_old, y_old), (x,y), (0,0,0), 4)
            x_old = x
            y_old = y

    cv2.imwrite(f'{name}_paper.jpg', image)

In [ ]:
#@title averagemeter.py

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
#@title eval.py

#!/usr/bin/python3.7
# adapted from: https://github.com/colincsl/TemporalConvolutionalNetworks/blob/master/code/metrics.py

import numpy as np
from pathlib import Path
import torch

class Metric():
    def __init__(self, run_type):
        self.run_type = run_type
        self.overlap = list(np.arange(0.4, 0.76, 0.05))
        self.thresholds_b = list(range(1, 5))

        self.correct = 0
        self.total = 0
        self.width_gt = []
        self.width_pred = []

        self.iou = []
        self.num_det, self.num_gt = 0, 0
        self.dist = []
        self.tp_list = np.zeros(len(self.overlap))
        self.fp_list = np.zeros(len(self.overlap))
        self.fn_list = np.zeros(len(self.overlap))

        self.tp_list_b = np.zeros(len(self.thresholds_b))
        self.fp_list_b = np.zeros(len(self.thresholds_b))
        self.fn_list_b = np.zeros(len(self.thresholds_b))

        self.tp_listm = np.zeros(len(self.overlap))
        self.fp_listm = np.zeros(len(self.overlap))
        self.fn_listm = np.zeros(len(self.overlap))
        self.accs = []
        self.f1percent = []

    def calc_scores_per_batch(self, pred, gt, gt_eval, mask=None):
        if mask is None:
            self.correct += (pred == gt_eval).int().sum().item()
            self.total += gt_eval.shape[1]
        else:
            self.correct += ((pred == gt).float()*mask[:, 0, :].squeeze(1)).sum().item()
            self.total += torch.sum(mask[:, 0, :]).item()

        pred = [item for sublist in torch_to_list(pred) for item in sublist]
        gt_eval = [item for sublist in torch_to_list(gt_eval) for item in sublist]
        gt = [item for sublist in torch_to_list(gt) for item in sublist]

        tp_list1, fp_list1, fn_list1, num_det1, num_gt1, dist1, width_pred1, width_gt1 = get_boundary_metric(pred, gt_eval, self.thresholds_b, bg_class=[0, -100])
        self.tp_list_b += tp_list1
        self.fp_list_b += fp_list1
        self.fn_list_b += fn_list1
        self.num_det += num_det1
        self.num_gt += num_gt1
        self.dist.extend(dist1)
        self.width_gt.append(width_gt1)
        self.width_pred.append(width_pred1)

        tp_list1, fp_list1, fn_list1, mean_iou = get_sign_metric(pred, gt, self.overlap, bg_class=[1, -100])
        self.iou.append(mean_iou)
        self.tp_list += tp_list1
        self.fp_list += fp_list1
        self.fn_list += fn_list1

        accs, f1percent = func_eval(gt,pred)

        self.accs.append(accs)
        self.f1percent.append(f1percent)



    def calc_metrics(self):
        self.mean_iou_ges = np.mean(self.iou)
        self.f1_sign = []
        for s in range(len(self.overlap)):
            self.precision = self.tp_list[s] / float(self.tp_list[s]+self.fp_list[s])
            self.recall = self.tp_list[s] / float(self.tp_list[s]+self.fn_list[s])

            f1 = 2.0 * (self.precision*self.recall) / (self.precision+self.recall)

            f1 = np.nan_to_num(f1)*100
            self.f1_sign.append(round(f1, 2))

        self.mean_f1s = np.mean(self.f1_sign)
        self.f1_sign = [self.f1_sign[2], self.f1_sign[-1]]

        self.f1b = []
        self.recall_list = []
        self.precision_list = []
        for s in range(len(self.thresholds_b)):
            self.precision_b = self.tp_list_b[s] / float(self.tp_list_b[s]+self.fp_list_b[s])
            self.recall_b = self.tp_list_b[s] / float(self.tp_list_b[s]+self.fn_list_b[s])

            f1_b = 2.0 * (self.precision_b*self.recall_b) / (self.precision_b+self.recall_b)

            f1_b = np.nan_to_num(f1_b)*100
            self.f1b.append(f1_b)
            self.recall_list.append(self.recall_b*100)
            self.precision_list.append(self.precision_b*100)

        self.mean_f1b = round(np.mean(self.f1b), 2)
        self.mean_recall_b = round(np.mean(self.recall_list), 2)
        self.mean_precision_b = round(np.mean(self.precision_list), 2)

        self.mean_dist = np.mean(np.abs(self.dist))
        self.mean_width_pred = np.mean(self.width_pred)
        self.mean_width_gt = np.mean(self.width_gt)
        self.mean_accs = np.mean(self.accs)
        self.mean_f1percent = np.mean(np.stack(self.f1percent), axis=0)


    def save_print_metrics(self, writer, save_dir, epoch, epoch_loss,video):

        writer.add_scalar(f'{self.run_type}/mF1B', self.mean_f1b, epoch+1)
        writer.add_scalar(f'{self.run_type}/mF1S', self.mean_f1s, epoch+1)

        #add to dict
        result_dict = {epoch: {}}
        result_dict[epoch] = {
            'Acc': 100*float(self.correct)/self.total, # Nuevo que agregué
            'mF1B': self.mean_f1b,
            'F1S': self.f1_sign[-1],
            'mF1S': self.mean_f1s,
            'IoU': 100*self.mean_iou_ges,
            'widthB': self.mean_width_pred,
            'dist': self.mean_dist,
            'detB': self.num_det,
            'mean f1@10':self.mean_f1percent[0],
            'mean f1@25':self.mean_f1percent[1],
            'mean f1@50':self.mean_f1percent[2],
            'mean accs':self.mean_accs
        }

        if self.run_type == 'train' or self.run_type == 'eval':
            print_str = f"[E{epoch + 1} / {self.run_type}]: epoch loss = {epoch_loss:.4f},   acc = {100*float(self.correct)/self.total:.2f}, mean F1B = {self.mean_f1b:.2f}, mean_F1S = {self.mean_f1s:.2f}, mean_F1@50 = {self.mean_f1percent}"
            save_str = f"[E{epoch + 1} / {self.run_type}]: epoch loss = {epoch_loss:.4f},   acc = {100*float(self.correct)/self.total:.2f}, F1_bound = {self.f1b}, mean_recall_b = {self.mean_recall_b}, mean_precision_b = {self.mean_precision_b}, mean F1B = {self.mean_f1b:.2f}, F1_sign = {self.f1_sign}, mean_F1S = {self.mean_f1s:.2f}, IoU_sign = {100*self.mean_iou_ges:.2f}, #B ({self.num_det}/{self.num_gt}), dist = {self.mean_dist}, boundary width: {self.mean_width_pred:.2f} / {self.mean_width_gt:.2f}  \n"

            if self.run_type == 'train':
                print(f'\n{print_str}')
            else:
                print(f'{print_str}\n')

            with open(f'{save_dir}/train_progress.txt', 'a+') as f:
                if self.run_type == 'train':
                    f.write('\n\n ---------------------------------------------------\n')
                f.write(save_str)

        elif self.run_type == 'test':
            # save_dir = f'{str(Path(save_dir).parent)}/{str(Path(save_dir).stem)}.txt'

            save_dir = f'{str(Path(save_dir))}/{str("results_test")}.txt'


            print(f"Acc: {(100*float(self.correct)/self.total):.2f}")
            print(f"F1B: {(self.f1b)}")
            print(f"mean Recall B: {(self.mean_recall_b)}")
            print(f"mean Precision B: {(self.mean_precision_b)}")
            print(f'mean F1B: {(self.mean_f1b):.2f}')
            print(f'IoU Sign: {(100*self.mean_iou_ges):.2f}')
            print(f'F1s: {(self.f1_sign)}')
            print(f'mean F1S: {(self.mean_f1s):.2f}')
            print(f'#B:  ({self.num_det}/{self.num_gt})')
            print(f'mean dist [frames]: {self.mean_dist:.2f}')
            print(f'mean B width pred/gt [frames]: {self.mean_width_pred:.2f} / {self.mean_width_gt:.2f} \n')
            print(f'mean accs: {self.mean_accs}')
            print(f'mean f1@10: {self.mean_f1percent[0]:.2f}')
            print(f'mean f1@25: {self.mean_f1percent[1]:.2f}')
            print(f'mean f1@50: {self.mean_f1percent[2]:.2f}')


            with open(save_dir, 'a') as f:
                f.write(f'video: {video} \n')
                f.write(f'Acc: {(100*float(self.correct)/self.total):.2f} \n')
                f.write(f'F1b: {(self.f1b)} \n')
                f.write(f'mean Recall B: {(self.mean_recall_b):.2f} \n')
                f.write(f'mean Precision B: {(self.mean_precision_b):.2f} \n')
                f.write(f'mean F1b: {(self.mean_f1b):.2f} \n')
                f.write(f'IoU Sign: {(100*self.mean_iou_ges):.2f} \n')
                f.write(f'F1s: {(self.f1_sign)} \n')
                f.write(f'mean F1S: {(self.mean_f1s):.2f} \n')
                f.write(f'#B:  ({self.num_det}/{self.num_gt})  \n')
                f.write(f'mean dist [frames]: {self.mean_dist:.2f} \n')
                f.write(f'mean B width pred/gt [frames]: {self.mean_width_pred:.2f} / {self.mean_width_gt:.2f} \n')
                f.write(f'mean accs: {self.mean_accs}')
                f.write(f'mean f1@10: {self.mean_f1percent[0]:.2f}\n')
                f.write(f'mean f1@25: {self.mean_f1percent[1]:.2f}\n')
                f.write(f'mean f1@50: {self.mean_f1percent[2]:.2f}\n')

        return result_dict


def get_boundary_metric(pred, gt, thresholds, bg_class=[0]):

    p_label, p_start, p_end = get_labels_start_end_time(pred, bg_class)
    y_label, y_start, y_end = get_labels_start_end_time(gt, bg_class)

    mean_boundary_width_pred = list(pred).count(1) / len(p_label) if len(p_label) else 0
    mean_boundary_width_gt = list(gt).count(1) / len(y_label) if len(y_label) else 0

    num_pred = len(p_label)
    num_gt = len(y_label)

    pos_p = [(p_end[i]+p_start[i])/2 for i in range(len(p_label))]
    pos_y = [(y_end[i]+y_start[i])/2 for i in range(len(y_label))]

    # calculate distance matrix
    if len(p_label) > 0:
        dist_all = []
        for p in pos_p:
            dist_all.append([abs(y-p) for y in pos_y])
        dist_arr = np.asarray(dist_all)

        # calculate mean distance
        mean_dist = [np.mean(np.min(dist_arr, 1))]

    else:
        mean_dist = [0]

    # find smallest distances
    dist_choosen = []
    if len(p_label) > 0:
        for ix in range(min(dist_arr.shape[0], dist_arr.shape[1])):

            argmin_row = np.argmin(dist_arr, axis=1)
            min_row = np.min(dist_arr, axis=1)
            min_dist = np.min(min_row)
            argmin_dist = np.argmin(min_row)

            dist_choosen.append(min_dist)

            # delete row and column -> pred-gt pair can't be reused
            dist_arr = np.delete(dist_arr, argmin_dist, 0)
            dist_arr = np.delete(dist_arr, argmin_row[argmin_dist], 1)

    tp_list = []
    fp_list = []
    fn_list = []

    for th in thresholds:
        tp = 0
        fp = 0
        for dist in dist_choosen:
            if dist <= th:
                tp += 1
            else:
                fp += 1

        # more predictions than gt -> count as false positiv
        fp += max(0, len(p_label)-len(dist_choosen))
        # difference between number of true boundaries and correct predicted ones -> false negative
        fn = len(y_label) - tp

        tp_list.append(tp)
        fp_list.append(fp)
        fn_list.append(fn)

    return np.asarray(tp_list), np.asarray(fp_list), np.asarray(fn_list), num_pred, num_gt, mean_dist, mean_boundary_width_pred, mean_boundary_width_gt


def get_sign_metric(pred, gt, overlap, bg_class=[1]):
    p_label, p_start, p_end = get_labels_start_end_time(pred, bg_class)
    y_label, y_start, y_end = get_labels_start_end_time(gt, bg_class)
    iou_all = []

    for j in range(len(p_label)):
        intersection = np.minimum(p_end[j], y_end) - np.maximum(p_start[j], y_start)
        union = np.maximum(p_end[j], y_end) - np.minimum(p_start[j], y_start)
        iou_all.append((1.0*intersection / union)*([p_label[j] == y_label[x] for x in range(len(y_label))]))

    iou_arr = np.asarray(iou_all)
    iou_choosen = []
    if len(p_label) > 0:
        for ix in range(min(iou_arr.shape[0], iou_arr.shape[1])):
            argmax_row = np.argmax(iou_arr, axis=1)
            max_row = np.max(iou_arr, axis=1)
            max_iou = np.max(max_row)
            argmax_iou = np.argmax(max_row)
            iou_choosen.append(max_iou)
            iou_arr = np.delete(iou_arr, argmax_iou, 0)
            iou_arr = np.delete(iou_arr, argmax_row[argmax_iou], 1)

        diff = max(iou_arr.shape[0], iou_arr.shape[1]) - len(iou_choosen)
    else:
        diff = len(y_label)

    tp_list = []
    fp_list = []
    fn_list = []

    for ol in overlap:
        tp = 0
        fp = 0
        for match in iou_choosen:
            if match > ol:
                tp += 1
            else:
                fp += 1
        fp += max(0, len(p_label)-len(iou_choosen))
        fn = len(y_label) - tp
        tp_list.append(tp)
        fp_list.append(fp)
        fn_list.append(fn)

    iou_choosen.extend([0]*diff)
    mean_iou = np.mean(iou_choosen)

    return np.asarray(tp_list), np.asarray(fp_list), np.asarray(fn_list), mean_iou


def func_eval(gt_content, pred_content):

          overlap = [.1, .25, .5]
          tp, fp, fn = np.zeros(3), np.zeros(3), np.zeros(3)

          correct = 0
          total = 0
          edit = 0

          assert(len(gt_content) == len(pred_content))

          for i in range(len(gt_content)):
              total += 1
              if gt_content[i] == pred_content[i]:
                  correct += 1

              # edit += edit_score(pred_content, gt_content)

          for s in range(len(overlap)):
                  tp1, fp1, fn1 = f_score_s(pred_content, gt_content, overlap[s],bg_class=[-100])
                  tp[s] += tp1
                  fp[s] += fp1
                  fn[s] += fn1


          acc = 100 * float(correct) / total
          f1s = np.array([0, 0 ,0], dtype=float)
          for s in range(len(overlap)):
              precision = tp[s] / float(tp[s] + fp[s])
              recall = tp[s] / float(tp[s] + fn[s])

              f1 = 2.0 * (precision * recall) / (precision + recall)

              f1 = np.nan_to_num(f1) * 100
              f1s[s] = f1


          return acc, f1s


def f_score_s(recognized, ground_truth, overlap, bg_class=[-100]):
          p_label, p_start, p_end = get_labels_start_end_time(recognized, bg_class)
          y_label, y_start, y_end = get_labels_start_end_time(ground_truth, bg_class)

          tp = 0
          fp = 0

          hits = np.zeros(len(y_label))

          for j in range(len(p_label)):
              intersection = np.minimum(p_end[j], y_end) - np.maximum(p_start[j], y_start)
              union = np.maximum(p_end[j], y_end) - np.minimum(p_start[j], y_start)
              IoU = (1.0*intersection / union)*([p_label[j] == y_label[x] for x in range(len(y_label))])
              # Get the best scoring segment
              idx = np.array(IoU).argmax()

              if IoU[idx] >= overlap and not hits[idx]:
                  tp += 1
                  hits[idx] = 1
              else:
                  fp += 1
          fn = len(y_label) - sum(hits)

          return float(tp), float(fp), float(fn)

In [ ]:
# #@title dataloader

# import pympi
# import sys
# import os
# import datetime
# import pickle

# def get_weights(gt_dict):
#     """Calculate weights for weighted cross entropy loss.

#     Args:
#         gt_dict: dictionary of the gt labels.

#     Returns:
#         list of weights per class
#     """
#     count_list = [0, 0]
#     for _, item in gt_dict.items():
#         item = list(item)
#         count_list[1] += item.count(1)
#         count_list[0] += item.count(0)

#     weights = [1 / count_list[i] if count_list[i] != 0 else 1 for i in range(2)]
#     weights_norm = [i / sum(weights) for i in weights]
#     return weights_norm


# class DataLoader():
#     def __init__(self, args, dataset, setname, checkpoint_ms=None, results_dir=None):
#         self.dataset = dataset
#         self.setname = setname
#         self.checkpoint_ms = checkpoint_ms
#         self.results_dir = results_dir

#         self.features_dict = {}
#         self.gt_dict = {}
#         self.eval_gt_dict = {}
#         self.CP_dict = None

#         # self.feature_path = f'/content/drive/MyDrive/Tesis/data/features/{self.dataset}/{args.i3d_training}'
#         self.feature_path = None


#         self.get_data(args)

#     def get_data(self, args):
#         # Dynamically load the dataset handler from the datasets module
#         # datasets = "/content/drive/MyDrive/Tesis/datasets/"
#         # dataloader = getattr(datasets, f'{self.dataset}')
#         # feature_loader = dataloader(args, self.feature_path, self.setname, self.results_dir)

#         #### Como todo está en el mismo notebook, no es necesario usar getattr
#         feature_loader = LSP_datasets(args, self.feature_path, self.setname, self.results_dir)

#         # Load Change Points (CP) dictionary if needed
#         # if args.eval_use_CP:
#         #     self.CP_dict = feature_loader.CP_dict

#         # Update the data with the loaded features, ground truths, and video list
#         self.features_dict = feature_loader.features_dict
#         self.gt_dict = feature_loader.gt_dict
#         self.eval_gt_dict = feature_loader.eval_gt_dict
#         self.vid_list = feature_loader.vid_list
#         self.num_classes = feature_loader.num_classes

#         # Determine the weights for training
#         self.weights = args.weights
#         if args.weights == 'opt':
#             self.weights = get_weights(self.gt_dict)

#@title dataloader

import pympi
import sys
import os
import datetime
import pickle

def get_weights(gt_dict):
    """Calculate weights for weighted cross entropy loss.

    Args:
        gt_dict: dictionary of the gt labels.

    Returns:
        list of weights per class
    """
    count_list = [0, 0]
    for _, item in gt_dict.items():
        item = list(item)
        count_list[1] += item.count(1)
        count_list[0] += item.count(0)

    weights = [1 / count_list[i] if count_list[i] != 0 else 1 for i in range(2)]
    weights_norm = [i / sum(weights) for i in weights]
    return weights_norm


class DataLoader():
    def __init__(self, args, dataset, setname,split_id, checkpoint_ms=None, results_dir=None):
        self.dataset = dataset
        self.setname = setname
        self.checkpoint_ms = checkpoint_ms
        self.results_dir = results_dir

        self.features_dict = {}
        self.gt_dict = {}
        self.eval_gt_dict = {}
        self.CP_dict = None

        # self.feature_path = f'/content/drive/MyDrive/Tesis/data/features/{self.dataset}/{args.i3d_training}'
        self.path_datasets=args.path_datasets

        self.list_of_datasets=args.list_of_datasets
        self.list_of_data_ver=args.list_of_data_ver
        self.joined_splits=args.joined_splits
        self.split_id=split_id




        self.get_data(args)

    def get_data(self, args):
        # Dynamically load the dataset handler from the datasets module
        # datasets = "/content/drive/MyDrive/Tesis/datasets/"
        # dataloader = getattr(datasets, f'{self.dataset}')
        # feature_loader = dataloader(args, self.feature_path, self.setname, self.results_dir)

        #### Como todo está en el mismo notebook, no es necesario usar getattr
        feature_loader = LSP_datasets(args,self.setname)

        # Load Change Points (CP) dictionary if needed
        # if args.eval_use_CP:
        #     self.CP_dict = feature_loader.CP_dict

        # Update the data with the loaded features, ground truths, and video list
        self.features_dict = feature_loader.features_dict
        self.gt_dict = feature_loader.gt_dict
        self.eval_gt_dict = feature_loader.eval_gt_dict
        self.vid_list = feature_loader.vid_list
        self.num_classes = feature_loader.num_classes

        # Determine the weights for training
        self.weights = args.weights
        if args.weights == 'opt':
            self.weights = get_weights(self.gt_dict)


In [ ]:
#@title preprocess.py
import pickle
import numpy as np
from math import ceil
from scipy.stats import norm
from pathlib import Path
import json

def gauss_dist(len_vector, mean, std):
    vec = np.round(norm.pdf(list(range(len_vector)), loc=mean, scale=std), 2)
    return vec/max(vec)


def find_boundaries(glosses, num_boundary_frames, boundaries_labels=None, uniform=0):
    boundaries = []
    signs = []
    for i, sen in enumerate(glosses):

        if len(sen) == 0:
            boundaries.append([0]*len(sen))
            continue
        boundaries.append([0]*len(sen))
        old_gloss = sen[0]
        for ix, gloss in enumerate(sen):
            if gloss != old_gloss:
                boundaries[-1] = [1 if i >= ix-round(num_boundary_frames/2) and i < ix+max(0, num_boundary_frames//2) else old for i,old in enumerate(boundaries[-1])]
            old_gloss = gloss
        if sum(boundaries[i]) == 0 and boundaries_labels != None:
            boundaries[i] = boundaries_labels[i]
        if uniform:
            signs.append((sum(boundaries[i][8:-7])//num_boundary_frames) + 1)   # hard coded in_frames 16 need to change

    if uniform:
        return boundaries, signs
    return boundaries

def dilate_boundaries(gt):
    eval_boundaries = []
    item=gt
    gt_temp = [0,0]+item+[0,0]
    con = 0
    for ix in range(2, len(item)+2):
            if con:
                con = 0
                continue
            if gt_temp[ix] == 1 and gt_temp[ix+1] == 0 and gt_temp[ix+2] == 0:
                gt_temp[ix+1] = 1
                con = 1
            if gt_temp[ix] == 1 and gt_temp[ix-1] == 0 and gt_temp[ix-2] == 0:
                gt_temp[ix-1] = 1
    eval_boundaries.append(gt_temp[2:-2])
    return eval_boundaries


def find_boundaries_eval(glosses, num_boundary_frames, boundaries_labels=None):
    # broader boundaries but restricted so that not two boundaries melted togehter
    boundaries = []

    for i, sen in enumerate(glosses):
        #count instance length

        if len(sen) == 0:
            boundaries.append([])
            continue
        old_gloss = sen[0]
        len_instances = []
        count_frames = 0
        for ix, gloss in enumerate(sen):

            if gloss == old_gloss:
                count_frames += 1
                old_gloss = gloss

            if gloss != old_gloss and ix == len(sen)-1:
                len_instances.append(count_frames)
                len_instances.append(1)

            elif gloss != old_gloss or ix == len(sen)-1:
                len_instances.append(count_frames)
                count_frames = 1
                old_gloss = gloss

        assert sum(len_instances)==len(sen)
        boundaries.append(np.zeros(len(sen), dtype=int))

        old_gloss = sen[0]
        counter = 0
        for ix, gloss in enumerate(sen):
            if gloss != old_gloss:
                counter += 1
                start = ix - min(num_boundary_frames//2, round(len_instances[counter-1]/4))
                end = ix + min(num_boundary_frames//2, round(len_instances[counter]/4))
                boundaries[-1][start:end] = 1

            old_gloss = gloss

        if sum(boundaries[-1]) == 0:
            boundaries[-1] = boundaries_labels[i]

    return boundaries

def filter_silence(glosses, threshold):
    filtered_glosses = []
    for seq in glosses:
        if -1 in seq:
            subseq = np.zeros(len(seq), dtype=int)
            count_silence = 0

            for ix, item in enumerate(seq):
                if item != -1:
                    if count_silence > threshold:
                        start = ix-count_silence
                        subseq[start:ix] = -1
                        count_silence = 0
                    elif count_silence != 0:
                        start = ix-count_silence
                        if ix == count_silence:
                            subseq[start:start+count_silence//2] = item
                        else:
                            subseq[start:start+count_silence//2] = previous_item
                        subseq[start+count_silence//2:ix] = item
                        count_silence = 0

                    subseq[ix] = item
                    previous_item = item

                else:
                    count_silence += 1
            filtered_glosses.append(subseq)

        else:
            filtered_glosses.append(seq)

    return filtered_glosses

In [ ]:
#@title \__init\__.py

__all__ = (
    "LSP_datasets"
)

In [ ]:
#@title batch_gen.py

"""import torch
import numpy as np
import random


class BatchGenerator(object):
    def __init__(self, num_classes, gt_dict, features_dict, gt_dict_eval=None):
        self.list_of_examples = list()
        self.index = 0
        self.num_classes = num_classes
        self.gt_dict = gt_dict
        self.features_dict = features_dict
        self.gt_dict_eval = gt_dict_eval

    def reset(self):
        self.index = 0
        random.shuffle(self.list_of_examples)

    def has_next(self):
        if self.index < len(self.list_of_examples):
            return True
        return False

    def get_max_index(self):
        return len(self.list_of_examples)

    def read_data(self, vid_list_file):
        self.list_of_examples = vid_list_file
        random.shuffle(self.list_of_examples)

    def next_batch(self, batch_size):
        batch = self.list_of_examples[self.index:self.index + batch_size]
        self.index += batch_size

        batch_input = []
        batch_target = []
        batch_target_eval = []
        for vid in batch:

            if vid not in self.features_dict:
                    print(f"Video ID {vid} not found in features_dict")
                    continue  # Skip if the video ID is not found

            features = np.swapaxes(self.features_dict[vid], 0, 1)
            classes = self.gt_dict[vid]

            batch_input.append(features)
            batch_target.append(np.asarray(classes))

            if self.gt_dict_eval is not None:
                classes_eval = self.gt_dict_eval[vid]
                batch_target_eval.append(np.asarray(classes_eval))

        # print(f"batch_target_eval_tensor[i].shape", len(batch_target_eval))

        shape_index = 0
        length_of_sequences = [item.shape[shape_index] for item in batch_target]
        batch_input_tensor = torch.zeros(len(batch), np.shape(batch_input[0])[0],  max(length_of_sequences), dtype=torch.float)

        # regression
        if self.num_classes == 1:
            batch_target_tensor = torch.ones(len(batch), max(length_of_sequences), dtype=torch.float)*(-100.)
            batch_target_eval_tensor = torch.ones(len(batch), max(length_of_sequences), dtype=torch.float)*(-100.)
            shape_index = 0

        # classification
        else:
            batch_target_tensor = torch.ones(len(batch), max(length_of_sequences), dtype=torch.long)*(-100)
            batch_target_eval_tensor = torch.ones(len(batch), max(length_of_sequences), dtype=torch.long)*(-100)
            shape_index = 0

        mask = torch.zeros(len(batch_input), self.num_classes, max(length_of_sequences), dtype=torch.float)

        ###--- El Nuevo ---###
        for i in range(len(batch_input)):
            # Fill the input and target tensors
            batch_input_tensor[i, :, :np.shape(batch_input[i])[1]] = torch.from_numpy(batch_input[i])
            batch_target_tensor[i, :np.shape(batch_target[i])[shape_index]] = torch.from_numpy(batch_target[i])
            mask[i, :, :np.shape(batch_target[i])[shape_index]] = torch.ones(self.num_classes, np.shape(batch_target[i])[shape_index])

            if self.gt_dict_eval is not None:
                # Ensure that batch_target_eval[i] has the correct shape by squeezing it if necessary
                target_eval_tensor = torch.from_numpy(batch_target_eval[i])

                # Squeeze only if there is a dimension that has a length of 1
                if target_eval_tensor.dim() == 2 and target_eval_tensor.size(0) == 1:
                    target_eval_tensor = target_eval_tensor.squeeze(0)

                # Now assign it to the tensor
                batch_target_eval_tensor[i, :target_eval_tensor.size(0)] = target_eval_tensor
        #######################

        ###-- El Original ---###
        for i in range(len(batch_input)):
            batch_input_tensor[i, :, :np.shape(batch_input[i])[1]] = torch.from_numpy(batch_input[i])
            batch_target_tensor[i, :np.shape(batch_target[i])[shape_index]] = torch.from_numpy(batch_target[i])
            mask[i, :, :np.shape(batch_target[i])[shape_index]] = torch.ones(self.num_classes, np.shape(batch_target[i])[shape_index])

            if self.gt_dict_eval is not None:
                print(f"batch_target_eval_tensor[i].shape: {batch_target_eval_tensor[i].shape}")
                print(f"batch_target_eval[i].shape: {torch.from_numpy(batch_target_eval[i]).shape}")
                batch_target_eval_tensor[i, :np.shape(batch_target_eval[i])[shape_index]] = torch.from_numpy(batch_target_eval[i]) # aqui el problema
        #########################

        if self.gt_dict_eval is not None:
            return batch_input_tensor, batch_target_tensor, batch_target_eval_tensor, mask
        else:
            return batch_input_tensor, batch_target_tensor, mask"""

import torch
import numpy as np
import random

class BatchGenerator(object):
    def __init__(self, num_classes, gt_dict, features_dict, gt_dict_eval=None):
        self.list_of_examples = list()
        self.index = 0
        self.num_classes = num_classes
        self.gt_dict = gt_dict
        self.features_dict = features_dict
        self.gt_dict_eval = gt_dict_eval

    def reset(self):
        self.index = 0
        random.shuffle(self.list_of_examples)

    def has_next(self):
        if self.index < len(self.list_of_examples):
            return True
        return False

    def get_max_index(self):
        return len(self.list_of_examples)

    def read_data(self, vid_list_file):
        self.list_of_examples = vid_list_file
        random.shuffle(self.list_of_examples)

    def next_batch(self, batch_size):
        #print(f"\n--- Generating next batch of size: {batch_size} ---\n")

        # Extract the batch from the list of examples
        batch = self.list_of_examples[self.index:self.index + batch_size]
        self.index += batch_size

        # Initialize the batch variables
        batch_input = []
        batch_target = []
        batch_target_eval = []

        for vid in batch:
            #print(f"\nProcessing video: {vid}")

            # Check if the video ID exists in the features dictionary
            if vid not in self.features_dict:
                print(f"Video ID {vid} not found in features_dict")
                continue  # Skip if the video ID is not found

            # Get the features and ground truth for the video
            features = np.swapaxes(self.features_dict[vid], 0, 1)
            classes = self.gt_dict[vid]

            #print(f"Features shape (after swapaxes): {features.shape}")
            #print(f"Classes shape: {np.asarray(classes).shape}")

            # Append to the batch lists
            batch_input.append(features)
            batch_target.append(np.asarray(classes))

            # If evaluation ground truth exists, get it as well
            if self.gt_dict_eval is not None:
                classes_eval = (self.gt_dict_eval[vid])
                #print(f"Eval Classes shape: {np.asarray(classes_eval).shape}")
                batch_target_eval.append(np.asarray(classes_eval))

        #print(f"\nLength of batch_input: {len(batch_input)}")
        #print(f"Length of batch_target: {len(batch_target)}")

        # Check the shape of the sequences
        shape_index = 0
        length_of_sequences = [item.shape[shape_index] for item in batch_target]
        #print(f"Length of sequences in batch_target: {length_of_sequences}")

        # Initialize the tensors for input, target, and evaluation target
        batch_input_tensor = torch.zeros(len(batch), np.shape(batch_input[0])[0], max(length_of_sequences), dtype=torch.float)
        #print(f"Initialized batch_input_tensor with shape: {batch_input_tensor.shape}")

        # Initialize the target tensors based on classification/regression mode
        if self.num_classes == 1:
            batch_target_tensor = torch.ones(len(batch), max(length_of_sequences), dtype=torch.float) * -100.
            batch_target_eval_tensor = torch.ones(len(batch), max(length_of_sequences), dtype=torch.float) * -100.
        else:
            batch_target_tensor = torch.ones(len(batch), max(length_of_sequences), dtype=torch.long) * -100
            batch_target_eval_tensor = torch.ones(len(batch), max(length_of_sequences), dtype=torch.long) * -100

        #print(f"Initialized batch_target_tensor with shape: {batch_target_tensor.shape}")
        #if self.gt_dict_eval is not None:
        #    print(f"Initialized batch_target_eval_tensor with shape: {batch_target_eval_tensor.shape}")

        # Mask tensor initialization
        mask = torch.zeros(len(batch_input), self.num_classes, max(length_of_sequences), dtype=torch.float)
        #print(f"Initialized mask tensor with shape: {mask.shape}")

        # Populate the tensors with the batch data
        for i in range(len(batch_input)):
            batch_input_tensor[i, :, :np.shape(batch_input[i])[1]] = torch.from_numpy(batch_input[i])
            batch_target_tensor[i, :np.shape(batch_target[i])[shape_index]] = torch.from_numpy(batch_target[i])
            mask[i, :, :np.shape(batch_target[i])[shape_index]] = torch.ones(self.num_classes, np.shape(batch_target[i])[shape_index])

            #print(f"\nBatch input tensor {i} filled up to index: {np.shape(batch_input[i])[1]}")
            #print(f"Batch target tensor {i} filled up to index: {np.shape(batch_target[i])[shape_index]}")

            if self.gt_dict_eval is not None:
                #print(f"batch_target_eval_tensor[i].shape: {batch_target_eval_tensor[i].shape}")
                #print(f"batch_target_eval[i].shape: {torch.from_numpy(batch_target_eval[i]).shape}")
                #target_eval_tensor = torch.from_numpy(batch_target_eval[i]).squeeze()
                batch_target_eval_tensor[i, :np.shape(batch_target_eval[i])[shape_index]] = torch.from_numpy(batch_target_eval[i])

        # Return the populated tensors
        if self.gt_dict_eval is not None:
            return batch_input_tensor, batch_target_tensor, batch_target_eval_tensor, mask
        else:
            return batch_input_tensor, batch_target_tensor, mask

In [ ]:
# #@title LSP_datasets.py

# import random
# from pathlib import Path
# import pickle
# import numpy as np

# !pip install zsvision -q

# from zsvision.zs_utils import loadmat

# class LSP_datasets():
#     def __init__(
#             self,
#             args,
#             features_path,
#             setname,
#             results_dir=None,
#             ):

#         self.setname = setname

#         root_path = "/content/drive/MyDrive/Tesis/datasets/manejar_conflictos"
#         set_path = os.path.join(root_path, "splits", setname + ".split1.bundle")

#         print(set_path)  # read the bundle of the subset
#         with open(set_path, 'r') as file:
#             lines = file.readlines()

#         # create list of video names
#         if args.action == 'predict' and args.test_subset != -1:
#             self.vid_list = random.choices(self.vid_list, k=args.test_subset)

#         else:
#             self.vid_list = [video_name.split('.txt\n')[0] for ix, video_name in enumerate(lines)]  # get list of videos inside the subset

#         self.get_features(args)

#     def get_features(self, args):
#         self.num_classes = 2

#         # classification
#         self.gt_dict = {}
#         self.eval_gt_dict = {}
#         self.features_dict = {}

#         root_path = "/content/drive/MyDrive/Tesis/datasets/manejar_conflictos"

#         for video_idx in self.vid_list:
#             # Ensure video_idx is treated as a string
#             video_idx = str(video_idx)

#             # Antes era groundTruth (ahí están preprocesados), ahora es labels
#             with open(os.path.join(root_path, "labels", video_idx + ".txt"), 'r') as file:
#                 lines = file.readlines()

#             gt = [1 if line.strip() == "ME" else 0 for line in lines]
#             gt_eval=dilate_boundaries(gt)
#             # print("gt_eval",gt_eval)
#             # print("gt ",gt )

#             # Store the ground truth using string keys
#             self.gt_dict[video_idx] = gt[int(args.num_in_frames / 2):-int(args.num_in_frames / 2) + 1]

#             # Store the dilated boundaries for evaluation using string keys
#             self.eval_gt_dict[video_idx] = gt_eval[0][int(args.num_in_frames / 2):-int(args.num_in_frames / 2) + 1]

#             # Load features and store them using string keys
#             features_data = loadmat(Path(os.path.join(root_path, "preprocessed_videos","original_data_16_noreplicate", video_idx + ".mat")))
#             features = features_data['preds']

#             # if args.feature_normalization:
#             #     features = np.nan_to_num((features - np.mean(features, axis=0)) / np.std(features))

#             self.features_dict[video_idx] = features


#@title LSP_datasets.py

#@title LSP_datasets.py

import random
from pathlib import Path
import pickle
import numpy as np

!pip install zsvision -q

from zsvision.zs_utils import loadmat

class LSP_datasets():
    def __init__(
            self,
            args,
            setname
            ):

        # self.setname = args.setname

        self.path_datasets=args.path_datasets
        self.list_of_datasets=args.list_of_datasets
        self.list_of_data_ver=args.list_of_data_ver
        self.split_id=args.split_id
        self.joined_splits=args.joined_splits
        self.setname=setname

        # root_path = "/content/drive/MyDrive/Tesis/datasets/manejar_conflictos"

        print("joined_splits",self.joined_splits)

        if self.joined_splits==True:

          set_path = os.path.join(self.path_datasets, "joined_splits", self.setname +f".split{self.split_id}.bundle")

        else:

          # set_path = os.path.join(self.path_datasets,self.list_of_datasets[0], "splits", self.setname +f".split{self.split_id}.bundle")
          set_path = os.path.join(self.path_datasets,self.list_of_datasets[0], "splits", self.setname +f".{self.split_id}_veri_plain.bundle")
          # set_path = os.path.join(self.path_datasets,self.list_of_datasets[0], "splits", self.setname +f".{self.split_id}_veri.bundle")






        print("set_path",set_path)  # read the bundle of the subset
        print(self.split_id)
        with open(set_path, 'r') as file:
            lines = file.readlines()

        # create list of video names
        # if args.action == 'predict' and args.test_subset != -1:
        #     self.vid_list = random.choices(self.vid_list, k=args.test_subset)

        # else:
        self.vid_list = [video_name.split('.txt\n')[0] for ix, video_name in enumerate(lines)]  # get list of videos inside the subset

        self.get_features(args)

    def get_features(self, args):
        self.num_classes = 2

        # classification
        self.gt_dict = {}
        self.eval_gt_dict = {}
        self.features_dict = {}

        # root_path = "/content/drive/MyDrive/Tesis/datasets/manejar_conflictos"

        print(self.path_datasets)

        for video_idx in self.vid_list:
            # Ensure video_idx is treated as a string
            video_idx = str(video_idx)

            if self.joined_splits==True:

              video_name, dataset,ver_of_data=video_idx.split('_')

              ver_of_data=ver_of_data[0] #eliminando el .txt

              feature_ver=self.list_of_data_ver[int(ver_of_data)] #getting ver of data for features

              dataset_dir=self.list_of_datasets[int(dataset)]

              feature_file_path=os.path.join(self.path_datasets,dataset_dir, "features",feature_ver, video_name + ".mat")

              label_file_path=os.path.join(self.path_datasets,dataset_dir, "labels", video_name + ".txt")

            elif (len(self.list_of_datasets)==1 and len(self.list_of_data_ver)>1):

              ver_of_data=video_idx.split('_')[1][0]

              video_name=video_idx.split('_')[0]

              feature_ver=self.list_of_data_ver[int(ver_of_data)] #getting ver of data for features

              feature_file_path=os.path.join(self.path_datasets,self.list_of_datasets[0], "features",feature_ver, video_name + ".mat")

              label_file_path=os.path.join(self.path_datasets,self.list_of_datasets[0], "labels", video_name + ".txt")


            else:

              feature_file_path=os.path.join(self.path_datasets,self.list_of_datasets[0],"features", self.list_of_data_ver[0], video_idx + ".mat")

              label_file_path=os.path.join(self.path_datasets,self.list_of_datasets[0], "labels", video_idx + ".txt")


            # Antes era groundTruth (ahí están preprocesados), ahora es labels
            with open(label_file_path, 'r') as file:
                lines = file.readlines()

            gt = [1 if line.strip() == "ME" else 0 for line in lines]
            gt_eval=dilate_boundaries(gt)
            # print("gt_eval",gt_eval)


            # Store the ground truth using string keys
            # self.gt_dict[video_idx] = gt[int(args.num_in_frames / 2):-int(args.num_in_frames / 2) + 1]
            self.gt_dict[video_idx] = gt


            # Store the dilated boundaries for evaluation using string keys
            # self.eval_gt_dict[video_idx] = gt_eval[0][int(args.num_in_frames / 2):-int(args.num_in_frames / 2) + 1]
            self.eval_gt_dict[video_idx] = gt_eval[0]


            # Load features and store them using string keys
            features_data = loadmat(Path(feature_file_path))
            features = features_data['preds']

            # if args.feature_normalization:
            #     features = np.nan_to_num((features - np.mean(features, axis=0)) / np.std(features))

            self.features_dict[video_idx] = features



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 10.1 MB/s eta 0:00:00


In [ ]:
#@title bslcp.py
import random
from pathlib import Path
import pickle
import numpy as np

!pip install zsvision -q

#from self_labelling.changepoint_detection import get_CP_dict
#from utils.preprocess import gauss_dist, dilate_boundaries


class bslcp():
    def __init__(
            self,
            args,
            features_path,
            setname,
            results_dir = None,
            ):

        self.setname = setname
        self.features_path = f'{features_path}/{self.setname}'
        if setname == 'train':
            self.setvalue = 0
        elif setname == 'eval':
            self.setvalue = 1
        elif setname == 'test':
            self.setvalue = 2

        # load info file
        self.info_file = f'data/info/bslcp/info.pkl'
        with open(self.info_file, 'rb') as f:
            self.info_data = pickle.load(f)

        # create list of video names
        self.vid_list = [video_name for ix, video_name in enumerate(self.info_data['videos']['name']) if self.info_data['videos']['split'][ix] == self.setvalue]

        self.get_features(args)


    def get_features(self, args):
        self.num_classes = 2

        # classification
        gt_list = [glosses for ix, glosses in enumerate(self.info_data['videos']['alignments']['boundaries']) if self.info_data['videos']['split'][ix] == self.setvalue]
        gt_list_eval = dilate_boundaries(gt_list)

        # regression
        #if args.regression:
        #    self.num_classes = 1
        #    if self.setname == 'train':
        #        std = args.std
        #        gt_gauss = []
        #        for i, gt in enumerate(gt_list):
        #            label_old = gt[0]
        #            boundary_frames = []
        #            gauss = []
        #            for ix, label in enumerate(gt):
        #                if label == 1:
        #                    boundary_frames.append(ix)
        #                else:
        #                    if label_old:
        #                        gauss.append(gauss_dist(len(gt), np.mean(np.asarray(boundary_frames)), std))
        #                    boundary_frames = []
        #                label_old = label

        #            gauss = np.asarray(gauss)
        #            gt_gauss.append(list(np.amax(gauss, axis=0)))
        #        gt_list = gt_gauss.copy()
        #        gt_list_eval = dilate_boundaries([[int(round(item)) for item in sublist] for sublist in gt_list])


        # read i3d features
        features_data = loadmat(Path(f'{self.features_path}/features.mat'))
        features = features_data['preds']

        if args.feature_normalization:
            features = np.nan_to_num((features - np.mean(features, axis=0)) / np.std(features))

        # see if length of extracted features fit to the videos (input window size of i3d training (args.num_in_frames) leads to less features than frames)
        num_frames_list = [max(t-(args.num_in_frames-1), 0) for ix, t in enumerate(self.info_data['videos']['videos']['T']) if self.info_data['videos']['split'][ix] == self.setvalue]
        ges_frames = sum(num_frames_list)
        assert ges_frames == features.shape[0]
        assert args.features_dim == features.shape[1]

        # assign features to videos
        self.features_dict = {}
        for ix, video in enumerate(self.vid_list):
            start = sum(num_frames_list[:ix])
            end = start + num_frames_list[ix]
            self.features_dict[video] = features[start:end]

        # assign ground truth to videos
        self.gt_dict = {}
        for ix, video in enumerate(self.vid_list):
            self.gt_dict[video] = gt_list[ix][int(args.num_in_frames/2):-int(args.num_in_frames/2)+1]
            assert len(self.gt_dict[video]) == len(self.features_dict[video])

        self.eval_gt_dict = {}
        for ix, video in enumerate(self.vid_list):
            self.eval_gt_dict[video] = gt_list_eval[ix][int(args.num_in_frames/2):-int(args.num_in_frames/2)+1]
            assert len(self.eval_gt_dict[video]) == len(self.features_dict[video])

        # delete videos with less than 16 frames:
        del_list = np.where(np.asarray(num_frames_list) == 0)
        for item in del_list[0]:
            del self.gt_dict[self.vid_list[item]]
            del self.features_dict[self.vid_list[item]]
        self.vid_list = [vid for ix, vid in enumerate(self.vid_list) if ix not in list(del_list[0])]

        # choose random subset for prediction
        if args.action == 'predict' and args.test_subset != -1:
            self.vid_list = random.choices(self.vid_list, k=args.test_subset)

        if args.eval_use_CP:
            self.CP_dict = get_CP_dict(self.test_features_dict, self.test_vid_list, self.test_gt_dict)

### Definiendo el modelo

In [ ]:
#@title model.py

import os
import time
import copy
import pickle
import json
from math import ceil
from pathlib import Path
import datetime
from tqdm import tqdm

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class MultiStageModel(nn.Module):
    def __init__(self, num_stages, num_layers, num_f_maps, dim, num_classes):
        super(MultiStageModel, self).__init__()
        self.num_classes = num_classes
        self.stage1 = SingleStageModel(num_layers, num_f_maps, dim, num_classes)
        self.stages = nn.ModuleList([copy.deepcopy(SingleStageModel(num_layers, num_f_maps, num_classes, num_classes)) for s in range(num_stages-1)])

    def forward(self, x, mask):
        out = self.stage1(x, mask)
        outputs = out.unsqueeze(0)
        for s in self.stages:
            if self.num_classes == 1:
                out = s(torch.sigmoid(out) * mask[:, 0:1, :], mask)
            else:
                out = s(F.softmax(out, dim=1) * mask[:, 0:1, :], mask)
            outputs = torch.cat((outputs, out.unsqueeze(0)), dim=0)
        return outputs


class SingleStageModel(nn.Module):
    def __init__(self, num_layers, num_f_maps, dim, num_classes):
        super(SingleStageModel, self).__init__()
        self.num_classes = num_classes
        self.conv_1x1 = nn.Conv1d(dim, num_f_maps, 1)
        self.layers = nn.ModuleList([copy.deepcopy(DilatedResidualLayer(2 ** i, num_f_maps, num_f_maps)) for i in range(num_layers)])
        self.conv_out = nn.Conv1d(num_f_maps, num_classes, 1)

    def forward(self, x, mask):
        out = self.conv_1x1(x)
        for layer in self.layers:
            out = layer(out, mask)
        out = self.conv_out(out) * mask[:, 0:1, :]
        return out


class DilatedResidualLayer(nn.Module):
    def __init__(self, dilation, in_channels, out_channels):
        super(DilatedResidualLayer, self).__init__()
        self.conv_dilated = nn.Conv1d(in_channels, out_channels, 3, padding=dilation, dilation=dilation)
        self.conv_1x1 = nn.Conv1d(out_channels, out_channels, 1)
        self.dropout = nn.Dropout()

    def forward(self, x, mask):
        out = F.relu(self.conv_dilated(x))
        out = self.conv_1x1(out)
        out = self.dropout(out)
        return (x + out) * mask[:, 0:1, :]


class Trainer:
    def __init__(self, num_blocks, num_layers, num_f_maps, dim, num_classes, device, weights, save_dir):
        self.model = MultiStageModel(num_blocks, num_layers, num_f_maps, dim, num_classes)
        if weights is None:
            self.ce = nn.CrossEntropyLoss(ignore_index=-100)
        else:
            self.ce = nn.CrossEntropyLoss(weight=torch.tensor(weights).to(device), ignore_index=-100)

        self.mse = nn.MSELoss(reduction='none')
        self.mse_red = nn.MSELoss(reduction='mean')
        self.sm = nn.Softmax(dim=1)
        self.num_classes = num_classes
        self.writer = SummaryWriter(log_dir=f'{save_dir}/logs')
        self.global_counter = 0
        self.train_result_dict = {}
        self.test_result_dict = {}



    def train(self, save_dir, batch_gen, num_epochs, batch_size, learning_rate, device, eval_args, pretrained='',layer_to_unfreeze=None):

        self.train_losses = []
        self.val_losses = []
        self.train_Acc = []
        self.val_Acc = []
        self.train_mF1B = []
        self.val_mF1B = []
        self.train_mF1S = []
        self.val_mF1S = []
        self.train_mF110 = []
        self.val_mF110 = []
        self.train_mF125 = []
        self.val_mF125 = []
        self.train_mF150 = []
        self.val_mF150 = []
        self.train_mAcss = []
        self.val_mAcss = []


        self.model.train()
        self.model.to(device)

        # Load pretrained model
        if pretrained != '':
            pretrained_dict = torch.load(pretrained)
            # self.model.load_state_dict(pretrained_dict)
                    # Get the current model's state_dict
            model_dict = self.model.state_dict()

            # Filter out mismatched layers by size
            matched_weights = {}
            for name, weight in pretrained_dict.items():
                if name in model_dict:
                    # Check if the size matches
                    if model_dict[name].size() == weight.size():
                        matched_weights[name] = weight
                    # else:
                    #     print(f"Skipping layer '{name}' due to size mismatch: "
                    #           f"pretrained size {weight.size()}, model size {model_dict[name].size()}")

            # Update model's state_dict with the matched weights
            model_dict.update(matched_weights)

            # Load the updated state_dict into the model
            self.model.load_state_dict(model_dict)

        for name, param in self.model.named_parameters():
            # if any(layer in name for layer in layer_to_unfreeze):  # Partial matching logic
                param.requires_grad = True
            #     print(f"Unfrozen layer: {name}")  # Debugging
            # else:
            #     param.requires_grad = False










        # Filter parameters for the optimizer that require gradients
        params_to_optimize = filter(lambda p: p.requires_grad, self.model.parameters())

        # Initialize optimizer with only the parameters that require gradients
        optimizer = optim.Adam(params_to_optimize, lr=learning_rate)
        # optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            # Initialize the total loss for the current epoch
            epoch_loss = 0
            end = time.time()

            batch_time = AverageMeter()
            data_time = AverageMeter()

            # Progress bar using tqdm
            with tqdm(range(batch_gen.get_max_index()), desc=f"E{epoch + 1}") as bar:
                count = 0
                get_metrics_train = Metric('train')

                while batch_gen.has_next():
                    self.global_counter += 1

                    batch_input, batch_target, batch_target_eval, mask = batch_gen.next_batch(batch_size)
                    batch_input, batch_target, batch_target_eval, mask = batch_input.to(device), batch_target.to(device), batch_target_eval.to(device), mask.to(device)

                    print(batch_input.shape,batch_target.shape)
                    optimizer.zero_grad()
                    predictions = self.model(batch_input, mask)

                    loss = 0
                    for ix, p in enumerate(predictions):
                        if self.num_classes == 1:
                            loss += self.mse_red(p.transpose(2, 1).contiguous().view(-1, self.num_classes).squeeze(), batch_target.view(-1))
                        else:
                            loss += self.ce(p.transpose(2, 1).contiguous().view(-1, self.num_classes), batch_target.view(-1))
                            loss += 0.15 * torch.mean(torch.clamp(self.mse(F.log_softmax(p[:, :, 1:], dim=1), F.log_softmax(p.detach()[:, :, :-1], dim=1)), min=0, max=16) * mask[:, :, 1:])

                    epoch_loss += loss.item()
                    loss.backward()
                    optimizer.step()

                    if self.num_classes == 1:
                        predicted = torch.round(predictions[-1].data.squeeze())
                        gt = torch.round(batch_target)
                        gt_eval = batch_target_eval
                    else:
                        _, predicted = torch.max(predictions[-1].data, 1)
                        gt = batch_target
                        gt_eval = batch_target_eval

                    get_metrics_train.calc_scores_per_batch(predicted, gt, gt_eval, mask)

                    batch_time.update(time.time() - end)
                    end = time.time()

                    # Update tqdm progress bar postfix
                    bar.set_postfix(
                        Batch=f"{batch_time.avg:.1f}s",
                        Loss=loss.item(),
                    )
                    count += 1
                    bar.update()

            # Reset the batch generator for the next epoch
            batch_gen.reset()

            torch.save(self.model.state_dict(), save_dir + "/epoch-" + str(epoch + 1) + ".model")
            # torch.save(optimizer.state_dict(), save_dir + "/epoch-" + str(epoch + 1) + ".opt")

            get_metrics_train.calc_metrics()
            result_dict = get_metrics_train.save_print_metrics(self.writer, save_dir, epoch, epoch_loss/(len(batch_gen.list_of_examples)/batch_size))

            # Guardar resultados para plotear
            self.train_losses.append(epoch_loss/(len(batch_gen.list_of_examples)/batch_size))
            self.train_Acc.append(result_dict[epoch]['Acc'])
            self.train_mF1B.append(result_dict[epoch]['mF1B'])
            self.train_mF1S.append(result_dict[epoch]['mF1S'])
            self.train_mF110.append(result_dict[epoch]['mean f1@10'])
            self.train_mF125.append(result_dict[epoch]['mean f1@25'])
            self.train_mF150.append(result_dict[epoch]['mean f1@50'])
            self.train_mAcss.append(result_dict[epoch]['mean accs'])




            self.train_result_dict.update(result_dict)

            eval_args[7] = epoch
            eval_args[1] = save_dir + "/epoch-" + str(epoch+1) + ".model"
            self.predict(*eval_args)

        with open(f'{save_dir}/train_results.json', 'w') as fp:
            json.dump(self.train_result_dict, fp, indent=4)

        with open(f'{save_dir}/eval_results.json', 'w') as fp:
            json.dump(self.test_result_dict, fp, indent=4)

        self.writer.close()

    def predict(
            self,
            args,
            model_dir,
            results_dir,
            features_dict,
            gt_dict,
            gt_dict_dil,
            vid_list_file,
            epoch,
            device,
            mode,
            classification_threshold,
            uniform=0,
        ):

        ##########
        if mode == 'test':

            self.val_losses = []
            self.val_Acc = []
            self.val_mF1B = []
            self.val_mF1S = []
            self.val_mAcss = []
            self.val_mF110 = []
            self.val_mF125 = []
            self.val_mF150 = []

        #########
        save_score_dict = {}
        metrics_per_signer = {}
        get_metrics_test = Metric(mode)

        self.model.eval()
        with torch.no_grad():
            self.model.to(device)
            self.model.load_state_dict(torch.load(model_dir))

            epoch_loss = 0

            for vid in tqdm(vid_list_file):
                features = np.swapaxes(features_dict[vid], 0, 1)
                input_x = torch.tensor(features, dtype=torch.float).unsqueeze_(0).to(device)
                predictions = self.model(input_x, torch.ones(input_x.size(), device=device))

                if self.num_classes == 1:
                    pred_prob = predictions[-1].squeeze()
                    pred_prob = torch_to_list(pred_prob)
                    predicted = torch.tensor(np.where(np.asarray(pred_prob) > args.classification_threshold, 1, 0)).to(device)
                    gt = torch.tensor(gt_dict[vid]).to(device)
                    gt_eval = torch.tensor(gt_dict_dil[vid]).to(device)
                else:
                    pred_prob = torch_to_list(self.sm(predictions[-1]))[0][1]
                    predicted = torch.tensor(np.where(np.asarray(pred_prob) > args.classification_threshold, 1, 0)).to(device)
                    gt = torch.tensor(gt_dict[vid]).to(device)
                    gt_eval = torch.tensor(gt_dict_dil[vid]).to(device)

                if uniform:
                    num_signs = get_num_signs(gt_dict[vid])
                    len_clip = len(gt_dict[vid])
                    predicted = [0] * len_clip
                    dist_uni = len_clip / num_signs
                    for i in range(1, num_signs):
                        predicted[round(i*dist_uni)] = 1
                        predicted[round(i*dist_uni)+1] = 1
                    pred_prob = predicted
                    predicted = torch.tensor(predicted).to(device)

                loss = 0
                mask = torch.ones(self.num_classes, np.shape(gt)[0]).to(device)

                for ix, p in enumerate(predictions):
                    if self.num_classes == 1:
                        loss += self.mse_red(p.transpose(2, 1).contiguous().view(-1, self.num_classes).squeeze(), gt.view(-1))
                    else:
                        loss += self.ce(p.transpose(2, 1).contiguous().view(-1, self.num_classes), gt.view(-1))
                        loss += 0.15 * torch.mean(torch.clamp(self.mse(F.log_softmax(p[:, :, 1:], dim=1), F.log_softmax(p.detach()[:, :, :-1], dim=1)), min=0, max=16) * mask[:, 1:])

                epoch_loss += loss.item()

                cut_endpoints = True
                if cut_endpoints:
                    if sum(predicted[-2:]) > 0 and sum(gt_eval[-4:]) == 0:
                        for j in range(len(predicted)-1, 0, -1):
                            if predicted[j] != 0:
                                predicted[j] = 0
                            elif predicted[j] == 0 and j < len(predicted) - 2:
                                break

                    if sum(predicted[:2]) > 0 and sum(gt_eval[:4]) == 0:
                        check = 0
                        for j, item in enumerate(predicted):
                            if item != 0:
                                predicted[j] = 0
                                check = 1
                            elif item == 0 and (j > 2 or check):
                                break

                get_metrics_test.calc_scores_per_batch(predicted.unsqueeze(0), gt.unsqueeze(0), gt_eval.unsqueeze(0))

                save_score_dict[vid] = {}
                save_score_dict[vid]['scores'] = np.asarray(pred_prob)
                save_score_dict[vid]['gt'] = torch_to_list(gt)
                save_score_dict[vid]['predicted'] = torch_to_list(predicted.unsqueeze(0))


                if mode == 'test' and args.viz_results:
                      if not isinstance(vid, int):
                          f_name = vid.split('/')[-1].split('.')[0]
                      else:
                          f_name = str(vid)

                      viz_results_paper(
                          gt,
                          torch_to_list(predicted),
                          name=results_dir + "/" + f'{f_name}',
                          pred_prob=pred_prob,
                      )

                if mode == 'test':
                        pickle.dump(save_score_dict, open(f'{results_dir}/scores.pkl', "wb"))

                get_metrics_test.calc_metrics()
                save_dir = results_dir
                result_dict = get_metrics_test.save_print_metrics(self.writer, save_dir, epoch, epoch_loss / len(vid_list_file),vid)


                self.val_losses.append(epoch_loss / len(vid_list_file)) # Guardar Loss de validación
                self.val_Acc.append(result_dict[epoch]['Acc'])
                self.val_mF1B.append(result_dict[epoch]['mF1B'])
                self.val_mF1S.append(result_dict[epoch]['mF1S'])
                self.val_mF110.append(result_dict[epoch]['mean f1@10'])
                self.val_mF125.append(result_dict[epoch]['mean f1@25'])
                self.val_mF150.append(result_dict[epoch]['mean f1@50'])
                self.val_mAcss.append(result_dict[epoch]['mean accs'])


                self.test_result_dict.update(result_dict)


                with open(f'{results_dir}/eval_results.json', 'w') as fp:
                            json.dump(self.test_result_dict, fp, indent=4)


### Cargar el modelo

In [ ]:
#@title main.py

import random
import torch
import numpy as np
from pathlib import Path

# from utils.utils import create_folders
# from model import Trainer
# from datasets.dataloader import DataLoader
# from batch_gen import BatchGenerator

def main(args, device, model_load_dir, model_save_dir, results_save_dir):

    if args.action == 'train':
        # Load train dataset and test dataset
        print(f'Load train data: {args.train_data}')
        # train_loader = DataLoader(args, args.train_data, 'train')
        train_loader = DataLoader(args, args.train_data, 'train', args.split_id)
        print(f'Load validation data: {args.test_data}')
        # test_loader = DataLoader(args, args.test_data, 'validation')
        test_loader = DataLoader(args, args.test_data, 'validation', args.split_id)

        print(f'Start training.')
        trainer = Trainer(
            args.num_stages,
            args.num_layers,
            args.num_f_maps,
            args.features_dim,
            train_loader.num_classes,
            device,
            train_loader.weights,
            model_save_dir
        )

        eval_args = [
            args,
            model_save_dir,
            results_save_dir,
            test_loader.features_dict,
            test_loader.gt_dict,
            test_loader.eval_gt_dict,
            test_loader.vid_list,
            args.num_epochs,
            device,
            'eval',
            args.classification_threshold,
        ]

        batch_gen = BatchGenerator(
            train_loader.num_classes,
            train_loader.gt_dict,
            train_loader.features_dict,
            train_loader.eval_gt_dict
        )

        batch_gen.read_data(train_loader.vid_list)
        trainer.train(
            model_save_dir,
            batch_gen,
            args.num_epochs,
            args.bz,
            args.lr,
            device,
            eval_args,
            pretrained=model_load_dir,
            layer_to_unfreeze=args.layer_to_unfreeze
        )

    else:
        print(f'Load test data: {args.test_data}')
        # test_loader = DataLoader(args, args.test_data, 'test',args.split_id)
        test_loader = DataLoader(args, args.test_data, 'validation',args.split_id)


        print(f'Start inference.')
        trainer = Trainer(
            args.num_stages,
            args.num_layers,
            args.num_f_maps,
            args.features_dim,
            test_loader.num_classes,
            device,
            test_loader.weights,
            save_dir=results_save_dir
        )

        trainer.predict(
            args,
            model_load_dir,
            results_save_dir,
            test_loader.features_dict,
            test_loader.gt_dict,
            test_loader.eval_gt_dict,
            test_loader.vid_list,
            args.num_epochs,
            device,
            'test',
            args.classification_threshold,
            uniform=args.uniform,
        )

    return trainer





In [ ]:

import os
def plot_graphs(train, val, metric,root_path,name_of_exp):
    """
    Plots the training and validation metrics stored in the Model class.

    Args:
        train (list): List of training metrics.
        val (list): List of validation metrics.
        metric (str): Name of the metric to plot
    """
    x_axis = range(1, len(train) + 1)
    plt.figure(figsize=(10, 5))
    plt.plot(x_axis, train, label='Entrenamiento '+ metric, linestyle = '--')
    plt.plot(x_axis, val, label='Validación '+ metric)
    plt.xlim(1, len(train))
    plt.xlabel('Épocas')
    plt.ylabel(metric)
    # plt.title('Training and Validation '+ metric)
    plt.legend()
    plt.grid(True)

    if not os.path.exists(os.path.join(root_path,'figures')):
        os.makedirs(os.path.join(root_path,'figures'))

    plt.savefig(os.path.join(root_path,'figures',name_of_exp+f'{metric}.pdf'))
    plt.show()


In [ ]:
def get_best_epoch(df, f1b_col, f1s_col, best_epochs_df, aux_df):
    """Selects the best epoch and appends it to another DataFrame."""
    # Calculate min(F1B, F1S) for each epoch to ensure both are high
    df['min_f1'] = df[[f1b_col, f1s_col]].min(axis=1)

    # Filter for epochs with the highest min(F1B, F1S)
    max_min_f1 = df['min_f1'].max()
    candidates = df[df['min_f1'] == max_min_f1]

    # If there's a tie, select the epoch with the highest max(F1B, F1S)
    if len(candidates) > 1:
        candidates['max_f1'] = candidates[[f1b_col, f1s_col]].max(axis=1)
        best_index = candidates['max_f1'].idxmax()  # Get the index of the best row
    else:
        best_index = candidates.index[0]  # Get the index directly if only one candidate

    best_row = df.loc[best_index]

    # Add the epoch number (index + 1) to the row
    best_row['epoch'] = best_index + 1

    # Update the aux_df with the best row
    aux_df.update(best_row)

    # Convert aux_df to DataFrame to ensure it's properly formatted
    aux_df = pd.DataFrame([aux_df])

    # Append the best row to the best_epochs_df if it exists
    if best_epochs_df is not None:
        best_epochs_df = pd.concat([best_epochs_df, aux_df], ignore_index=True)

    # Return the updated DataFrame
    return best_epochs_df


# Parámtros fijos, 10 splits, diff ver_data same dataset

In [ ]:
import pandas as pd

class Config:
    def __init__(self,num_stages,num_layers,num_f_maps,folder_name,repetition,split_id,pretrained):
        # General settings
        self.refresh = False
        self.seed = 0
        self.repetition=repetition

        self.action = 'test'  # Choose between 'train' and 'predict'
        self.extract_set = 'test'  # Choose between 'train' and 'test'
        self.setname="validation"
        self.train_data = 'LSP_datasets'  # Options: ['bslcp', 'phoenix14']
        self.test_data = 'LSP_datasets'  # Options: ['bslcp', 'phoenix14']


        self.i3d_training = 'i3d_kinetics_bslcp_981'
        self.num_in_frames = 16
        self.folder = folder_name  # Folder to save results

        # Visualization settings
        self.test_subset = 1  # Use only a subset of the test set for evaluation and visualization
        self.viz_results = False  # Save visualizations of results and gt for each video sequence

        # MS-TCN HYPERPARAMETERS
        self.num_stages = num_stages #4
        self.num_layers = num_layers #10
        self.num_f_maps = num_f_maps #64
        self.features_dim = 1024
        self.bz = 5
        self.num_classes = 2
        self.lr = 0.0005
        self.num_epochs = 15
        self.extract_epoch = 10 # epoch to test
        self.weights = 'opt'  # None, [1., 5.], 'opt'
        # self.pretrained="/content/drive/MyDrive/Feature_Extraction_and_Classification/Models/MSTCN/mstcn_phoenix14_i3d_phoenix14.model"
        self.pretrained=pretrained

        # self.save_dir="/content/drive/MyDrive/Tesis/set_16"
        self.save_dir="/content/drive/MyDrive/Tesis/"+str(folder_name)

        self.path_datasets="/content/drive/MyDrive/Tesis/datasets"

        self.uniform = 0
        self.regression = 0
        self.std = 1
        self.classification_threshold = 0.5

        # Other settings
        self.feature_normalization = 0
        self.num_boundary_frames = 2
        # self.list_of_datasets=["manejar_conflictos","ira_alegria"]
        self.list_of_datasets=["manejar_conflictos"]

        # self.list_of_data_ver=["set_16_only_replicated","set_16_rotated_from10_10","set_16_zoomedin_zoomout_0.8_1.2"]
        self.list_of_data_ver=["set_16_only_replicated"]

        self.joined_splits=False
        self.split_id=split_id

        # self.layer_to_unfreeze=["stages.2","stages.1","stages.0","stage1"]
        self.layer_to_unfreeze=[""]

        # self.split_id=1

if __name__ == "__main__":

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize configuration


    # param_grid={"num_stages":[4,6,8],"num_layers":[8,10,12],"num_f_maps":[32,64,128]}
    # param_grid={"num_stages":[8],"num_layers":[10,12],"num_f_maps":[32,64,128]}
    param_grid={"num_stages":4,"num_layers":8,"num_f_maps":128}

    # list_of_best_epochs=[3,13,4,9,11,10,3,10,9,4]
    list_of_best_epochs=[3,3,3,11,4,11,3,2,7,7]


    folder_name="set16_joined_splits_again"
    # folder_name="set16_10splits_augmented
    folder_name="set16_data_zoom_trans"



    list_of_splits=np.arange(0,10)



    param_keys=list(param_grid.keys())
    # param_values=param_grid.values()
    param_values = list(param_grid.values())

    train_best_epochs_df=pd.DataFrame()
    val_best_epochs_df=pd.DataFrame()

    train_aux_df={}
    val_aux_df={}

    number_repetitions=1

    print(param_values)


    for i in range(number_repetitions):
      for split_id in list_of_splits:

        # pretrained=f"/content/drive/MyDrive/Tesis/set16_10splits_augmented/4_8_128_1024_5_0.0005_weighted_opt_0_split{split_id}/epoch-{list_of_best_epochs[split_id]}.model"
        # pretrained=f"/content/drive/MyDrive/Tesis/set16_joined_splits/4_8_128_1024_5_0.0005_weighted_opt_0_split{split_id}/epoch-{list_of_best_epochs[split_id]}.model"
        pretrained=f"/content/drive/MyDrive/Tesis/set16_data_zoom_trans/4_8_128_1024_5_0.0005_weighted_opt_0_split{split_id}/epoch-{list_of_best_epochs[split_id]}.model"



        args = Config(num_stages=param_values[0],num_layers=param_values[1],num_f_maps=param_values[2],folder_name=folder_name,repetition=i,split_id=split_id,pretrained=pretrained)


        val_aux_df.update({param_keys[0]:param_values[0],param_keys[1]:param_values[1],param_keys[2]:param_values[2], "split_id": split_id})
        # train_aux_df.update({param_keys[0]:param_values[0],param_keys[1]:param_values[1],param_keys[2]:param_values[2]})


        print(vars(args))


        # Set seed
        seed = args.seed
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

        ##
        root_path="/content/drive/MyDrive/Tesis/"+ folder_name
        name_of_exp=f"hiperparameters_{param_values[0]}_{param_values[1]}_{param_values[2]}_{split_id}_{i}"
        ##



        # Create model folders and save args
        model_load_dir, model_save_dir, results_save_dir = create_folders(args)
        trainer = main(args, device, model_load_dir, model_save_dir, results_save_dir)


        ### Saving data


        losses_val=trainer.val_losses
        acc_val=trainer.val_Acc
        mF1B_val=trainer.val_mF1B
        mF1S_val=trainer.val_mF1S
        val_f150=trainer.val_mF150
        val_f125=trainer.val_mF125
        val_f110=trainer.val_mF110
        val_mAcss=trainer.val_mAcss



        dict_to_save_val = {
            'losses': losses_val,
            'acc': acc_val,
            'mF1B': mF1B_val,
            'mF1S': mF1S_val,
            'f150': val_f150,
            'f125': val_f125,
            'f110': val_f110,
            'mAcss': val_mAcss
        }
        # df_train = pd.DataFrame(dict_to_save)
        df_val = pd.DataFrame(dict_to_save_val)

        # train_best_epochs_df=get_best_epoch(df_train, 'mF1B', 'mF1S',train_best_epochs_df,train_aux_df)
        val_best_epochs_df=get_best_epoch(df_val, 'mF1B', 'mF1S',val_best_epochs_df,val_aux_df)

        if not os.path.exists(os.path.join(root_path,"data_for_plotting")):
          os.makedirs(os.path.join(root_path,"data_for_plotting"))
          print(os.path.join(root_path,"data_for_plotting"))
        #   print(f"Folder created: {os.path.join(root_path,"data_for_plotting")}")
        # else:
        #   print(f"Folder already exists: {os.path.join(root_path,"data_for_plotting")}")



        # df_train.to_csv(os.path.join(root_path,"data_for_plotting",name_of_exp+'_data_train.csv'))
        df_val.to_csv(os.path.join(root_path,"data_for_plotting",name_of_exp+'_data_val.csv'))




        ##Plotting


    # train_best_epochs_df.to_csv(os.path.join("/content/drive/MyDrive/Tesis",folder_name,'train_best_epochs_df.csv'), index=False)
    val_best_epochs_df.to_csv(os.path.join("/content/drive/MyDrive/Tesis",folder_name,'val_best_epochs_df.csv'), index=False)


[4, 8, 128]
{'refresh': False, 'seed': 0, 'repetition': 0, 'action': 'test', 'extract_set': 'test', 'setname': 'validation', 'train_data': 'LSP_datasets', 'test_data': 'LSP_datasets', 'i3d_training': 'i3d_kinetics_bslcp_981', 'num_in_frames': 16, 'folder': 'set16_data_zoom_trans', 'test_subset': 1, 'viz_results': False, 'num_stages': 4, 'num_layers': 8, 'num_f_maps': 128, 'features_dim': 1024, 'bz': 5, 'num_classes': 2, 'lr': 0.0005, 'num_epochs': 15, 'extract_epoch': 10, 'weights': 'opt', 'pretrained': '/content/drive/MyDrive/Tesis/set16_data_zoom_trans/4_8_128_1024_5_0.0005_weighted_opt_0_split0/epoch-3.model', 'save_dir': '/content/drive/MyDrive/Tesis/set16_data_zoom_trans', 'path_datasets': '/content/drive/MyDrive/Tesis/datasets', 'uniform': 0, 'regression': 0, 'std': 1, 'classification_threshold': 0.5, 'feature_normalization': 0, 'num_boundary_frames': 2, 'list_of_datasets': ['manejar_conflictos'], 'list_of_data_ver': ['set_16_only_replicated'], 'joined_splits': False, 'split_id':

  0%|          | 0/35 [00:00<?, ?it/s]

Acc: 76.74
F1B: [np.float64(57.14285714285713), np.float64(57.14285714285713), np.float64(76.1904761904762), np.float64(76.1904761904762)]
mean Recall B: 63.64
mean Precision B: 70.0
mean F1B: 66.67
IoU Sign: 57.77
F1s: [np.float64(57.14), np.float64(9.52)]
mean F1S: 52.38
#B:  (10/11)
mean dist [frames]: 2.45
mean B width pred/gt [frames]: 9.30 / 7.73 

mean accs: 74.4186046511628
mean f1@10: 90.48
mean f1@25: 85.71
mean f1@50: 52.38


 14%|█▍        | 5/35 [00:02<00:09,  3.28it/s]

Acc: 78.50
F1B: [np.float64(57.777777777777786), np.float64(62.222222222222236), np.float64(71.1111111111111), np.float64(71.1111111111111)]
mean Recall B: 67.05
mean Precision B: 64.13
mean F1B: 65.56
IoU Sign: 62.21
F1s: [np.float64(71.11), np.float64(17.78)]
mean F1S: 57.22
#B:  (23/22)
mean dist [frames]: 3.26
mean B width pred/gt [frames]: 9.69 / 9.77 

mean accs: 76.5843023255814
mean f1@10: 91.07
mean f1@25: 88.69
mean f1@50: 63.69
Acc: 78.73
F1B: [np.float64(48.57142857142857), np.float64(62.857142857142854), np.float64(71.42857142857143), np.float64(71.42857142857143)]
mean Recall B: 63.57
mean Precision B: 63.57
mean F1B: 63.57
IoU Sign: 62.81
F1s: [np.float64(68.57), np.float64(20.0)]
mean F1S: 56.78
#B:  (35/35)
mean dist [frames]: 3.25
mean B width pred/gt [frames]: 9.54 / 9.72 

mean accs: 77.99222848641453
mean f1@10: 88.71
mean f1@25: 87.13
mean f1@50: 63.79
Acc: 78.24
F1B: [np.float64(41.30434782608695), np.float64(56.52173913043478), np.float64(67.3913043478261), np.f

...

 89%|████████▊ | 31/35 [00:03<00:00, 12.50it/s]

Acc: 77.42
F1B: [np.float64(37.078651685393254), np.float64(57.865168539325836), np.float64(73.03370786516852), np.float64(80.33707865168539)]
mean Recall B: 64.81
mean Precision B: 59.57
mean F1B: 62.08
IoU Sign: 63.43
F1s: [np.float64(73.7), np.float64(40.79)]
mean F1S: 64.10
#B:  (371/341)
mean dist [frames]: 3.64
mean B width pred/gt [frames]: 6.98 / 9.52 

mean accs: 78.10403956672278
mean f1@10: 85.91
mean f1@25: 81.86
mean f1@50: 63.11
Acc: 77.51
F1B: [np.float64(37.19676549865229), np.float64(58.22102425876011), np.float64(73.04582210242589), np.float64(80.05390835579514)]
mean Recall B: 65.3
mean Precision B: 59.25
mean F1B: 62.13
IoU Sign: 63.87
F1s: [np.float64(73.68), np.float64(41.03)]
mean F1S: 64.37
#B:  (389/353)
mean dist [frames]: 3.68
mean B width pred/gt [frames]: 6.97 / 9.62 

mean accs: 78.19690665969938
mean f1@10: 85.72
mean f1@25: 81.80
mean f1@50: 63.23
Acc: 77.68
F1B: [np.float64(37.615894039735096), np.float64(58.807947019867555), np.float64(73.3774834437086

100%|██████████| 35/35 [00:03<00:00,  9.39it/s]

Acc: 77.53
F1B: [np.float64(37.56345177664975), np.float64(58.883248730964475), np.float64(72.84263959390863), np.float64(79.94923857868021)]
mean Recall B: 65.47
mean Precision B: 59.44
mean F1B: 62.31
IoU Sign: 64.19
F1s: [np.float64(73.44), np.float64(40.66)]
mean F1S: 64.17
#B:  (413/375)
mean dist [frames]: 3.59
mean B width pred/gt [frames]: 7.08 / 9.65 

mean accs: 78.39131669728539
mean f1@10: 85.90
mean f1@25: 82.13
mean f1@50: 63.57
Acc: 77.33
F1B: [np.float64(37.5), np.float64(58.08823529411764), np.float64(71.56862745098039), np.float64(78.67647058823529)]
mean Recall B: 64.46
mean Precision B: 58.72
mean F1B: 61.46
IoU Sign: 63.52
F1s: [np.float64(72.88), np.float64(40.49)]
mean F1S: 63.77
#B:  (427/389)
mean dist [frames]: 3.67
mean B width pred/gt [frames]: 7.13 / 9.64 

mean accs: 78.25088001183275
mean f1@10: 85.67
mean f1@25: 81.80
mean f1@50: 63.27
Acc: 77.17
F1B: [np.float64(37.170263788968825), np.float64(57.79376498800958), np.float64(70.9832134292566), np.float64


<ipython-input-22-e865e45bc2de>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_row['epoch'] = best_index + 1
<ipython-input-22-e865e45bc2de>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_row['epoch'] = best_index + 1
